# Lending CLub Case Study

### Import Libraries

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Read csv file

In [ ]:
df1 = pd.read_csv("loan.csv")

print(df1.shape)
print(df1.dtypes)

#### Confirm if id has unique values

In [ ]:
repeated_values = df1['id'].value_counts()
repeated_values = repeated_values[repeated_values>1]
if repeated_values.empty:
    print("id has Unique values")
else:
    print("id has duplicate values")

#### Confirm if member id has unique values

In [ ]:
null_values = df1[df1['member_id'].isnull()]
repeated_values = df1['id'].value_counts()
repeated_values = repeated_values[repeated_values>1]
if repeated_values.empty:
    print("member_id has Unique values")
else:
    print("member_id has duplicate values")

## Univariate Analysis

In [ ]:
uni_df = df1.copy()
df1['int_rate'].describe()
uni_df['int_rate'].describe()

# Convert Strung into float data types
uni_df['int_rate'] = uni_df['int_rate'].str.replace("%","").astype(float)
uni_df['revol_util'] = uni_df['revol_util'].str.replace("%","").astype(float)


# Function to remove outliers
def remove_outliers(df, column_name):
    """
    Removes outliers from a specified column in a dataframe using the IQR method.

    Parameters:
    df (pd.DataFrame): The input dataframe.
    column_name (str): The name of the column to remove outliers from.

    Returns:
    pd.DataFrame: A new dataframe with outliers removed.
    """
    column = df[column_name]

    # Calculate the IQR
    Q1 = column.quantile(0.25)  # 25th percentile
    Q3 = column.quantile(0.75)  # 75th percentile
    IQR = Q3 - Q1

    # Define bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out rows where the column's values fall outside of the IQR bounds
    df_cleaned = df[(column >= lower_bound) & (column <= upper_bound)]

    return df_cleaned[column_name]


# Barplot for Grade
plt.figure(figsize=(4,4))
sns.barplot(x=uni_df['grade'].value_counts().sort_index().index, y=uni_df['grade'].value_counts().values)
plt.title('Grades')
plt.show()

# Barplot for Term
plt.figure(figsize=(4,4))
sns.barplot(x=uni_df['term'].value_counts().sort_index().index, y=uni_df['term'].value_counts().values)
plt.title('Term')
plt.show()


# Boxplot for loan_amnt
plt.figure(figsize=(4,4))
plt.boxplot(remove_outliers(uni_df, 'loan_amnt'))
plt.title('Loan Amount')
plt.show()

#Boxplot for annual income
plt.figure(figsize=(4,4))
plt.boxplot(remove_outliers(uni_df, 'annual_inc'))
plt.title('Annual Income')
plt.show()

print('Maximum annual income is :', uni_df['annual_inc'].max())

#Boxplot for interest rate
plt.figure(figsize=(4,4))
plt.boxplot(remove_outliers(uni_df, 'int_rate'))
plt.title('Interest Rate')
plt.show()

#Boxplot for dti
plt.figure(figsize=(4,4))
plt.boxplot(remove_outliers(uni_df, 'dti'))
plt.title('Debt To Income')
plt.show()

#Boxplot for installment
plt.figure(figsize=(4,4))
plt.boxplot(remove_outliers(uni_df, 'installment'))
plt.title('Installment')
plt.show()

#Boxplot for revolving line utilization rate
plt.figure(figsize=(4,4))
plt.boxplot(remove_outliers(uni_df, 'revol_util'))
plt.title('Revolving Line Utilization Rate')
plt.show()

## Conclusion on Univariate Analysis

##### Company tends to grant more loans to hhigh quality borrower
##### Shorter duration loans are more popular tthan longe duration

##### Majority of borrowers tend to choose loan amounts ranging from \$5k to \$1k

##### Annual income of borrowers is majorly concentrated between \$4 k to \$8 range


##### Nearly all the borrowers receive interest rate between 9 to 1%

##### Average Debt to Income ratio of borrowers is \~1%

##### Most of the borrowers have installments ranging froUSD \$1 to 400
##### Borrowers on average utilize 50% of their revolving ccredi lline00

## Bivariate and Multivariate Analysis to find drivers of Default and Recoveries

#### Exclude ‘Current’ Loans and calculate overall default rate

##### We will only analyze loans that are either charged off or paid off. Because Current loans don’t reflect the eventual quality of loan until it is either paid off or charged off

In [ ]:
df2 = df1[df1['loan_status'].isin(['Charged Off', 'Fully Paid'])]
print(df2.shape)
print(df2['loan_status'].value_counts())
print("Overall Default Rate is " + str(round(df2[df2[['loan_status']]=='Charged Off']['loan_status'].count() / df2['loan_status'].count() *100,1)) + "%")

## Default Rate by Grade

##### Grade - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'grade', 'loan_status', 'loan_amnt']]
sns.histplot(data=temp, x='grade', hue='loan_status', multiple='stack', stat='count')
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['grade','loan_status'])['id'].count()
loan_percentage = loan_count.groupby('grade').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

##### Grade - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'grade', 'loan_status', 'loan_amnt']]
sns.barplot(data=temp, x='grade', hue='loan_status', y='loan_amnt', estimator='sum')
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['grade','loan_status'])['loan_amnt'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('grade').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

## Conclusion on Grade

##### Lower Grade Borrowers are more likely to make defaults

##### It can be concluded that Grade is the primary driver that drives loan performance

## Default Rate by Term

##### Term - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'term', 'loan_status', 'loan_amnt']]
sns.histplot(data=temp, x='term', hue='loan_status', multiple='stack', stat='count')
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['term','loan_status'])['id'].count()
loan_percentage = loan_count.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)

##### Term - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'term', 'loan_status', 'loan_amnt']]
sns.barplot(data=temp, x='term', hue='loan_status', y='loan_amnt', estimator='sum')
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['term','loan_status'])['loan_amnt'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('term').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)

## Conclusion on Term

##### Default rates are noticeably higher in 5 yr term bucket compared to 3 yr bucket

##### But nothing can be concluded without analyzing the composition of term bucketse

## Relationship between Term and Grade

##### Term - Grade Relation by Loan Count

In [ ]:
temp = df2[['id', 'term', 'grade', 'loan_status', 'loan_amnt']]
sns.histplot(data=temp, x='term', hue='grade', multiple='stack', stat='count')
plt.show()

# Term - Grade composition by loan count
loan_count = temp.groupby(['term','grade'])['id'].count()
loan_percentage = loan_count.groupby('term').apply(lambda x: 100 * x / float(x.sum()))
print('\033[1;4m' + '\nComposition of Term Buckets by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nComposition of Term Buckets by Loan Count Percentage\n' + '\033[0m')
print(loan_percentage)

##### Term - Grade Relation by Loan Amount

In [ ]:
temp = df2[['id', 'term', 'grade', 'loan_status', 'loan_amnt']]
sns.barplot(data=temp, x='term', hue='grade', y='loan_amnt', estimator='sum')
plt.show()

# Term - Grade composition by loan amount
loan_amnt = temp.groupby(['term','grade'])['loan_amnt'].sum()
loan_percentage = loan_amnt.groupby('term').apply(lambda x: 100 * x / float(x.sum()))
print('\033[1;4m' + '\nComposition of Term Buckets by Loan Amount\n' + '\033[0m')
print(loan_amnt)
print('\033[1;4m' + '\nComposition of Term Buckets by Loan Amount Percentage\n' + '\033[0m')
print(loan_percentage)

## Conclusion on relationship between Term and Grade

##### Lower Grade Borrowers are more likely to make defaults

##### 5 year loans have higher default rates than 3 year loans

##### It is also found that 5 year loan has much higher composition of lower grade loans

##### Hence it can be concluded that Grade is the primary driver that drives loan performance

## Default Rate by Interest Rate

##### Interest Rate - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'int_rate', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())
temp['int_rate'] = temp['int_rate'].str.replace("%","").astype(float)

# Remove outliers

# Calculate the IQR (Interquartile Range)
Q1 = temp['int_rate'].quantile(0.25)  # 25th percentile
Q3 = temp['int_rate'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['int_rate'] >= lower_bound) & (temp['int_rate'] <= upper_bound)]
print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['int_rate'].describe())

sns.boxplot(temp['int_rate'])

# Create 4 buckets based on quartiles (percentile-based)
temp['bucket'], bin_intervals = pd.qcut(temp['int_rate'], q=6, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'], retbins=True)

# visualisation
plt.figure(figsize=(15,6))
sns.histplot(data=temp, x='bucket', hue='loan_status', multiple='stack', stat='count')
plt.xticks(rotation=45)
plt.show()

# Boxplot of annual income by loan status
plt.figure(figsize=(10,6))
sns.boxplot(x='loan_status', y='int_rate', data=temp)

# Set labels and title
plt.xlabel('Loan Status (Nominal)', fontsize=12)
plt.ylabel('int_rate (Continuous)', fontsize=12)
plt.title('Boxplot of int_rate by Loan Status', fontsize=14)

# Show the plot
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['bucket','loan_status'])['id'].count()
loan_percentage = loan_count.groupby('bucket').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

# Set x-axis labels as the 'income_bucket' values
plt.xlabel('int_rate Bucket', fontsize=12)
plt.ylabel('Charged Off Percentage (%)', fontsize=12)

# Set title
plt.title('Charged Off Loan Percentage by int_rate Bucket', fontsize=14)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


# Get the actual bin intervals (ranges)
print("\nBin Ranges:")
for i in range(len(bin_intervals) - 1):
    print(f"Bucket {i + 1}: {bin_intervals[i]} to {bin_intervals[i + 1]}")

##### Interest Rate - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'int_rate', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())
temp['int_rate'] = temp['int_rate'].str.replace("%","").astype(float)

# Remove outliers

# Calculate the IQR (Interquartile Range)
Q1 = temp['int_rate'].quantile(0.25)  # 25th percentile
Q3 = temp['int_rate'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['int_rate'] >= lower_bound) & (temp['int_rate'] <= upper_bound)]
print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['int_rate'].describe())

sns.boxplot(temp['int_rate'])

# Create 4 buckets based on quartiles (percentile-based)
temp['bucket'], bin_intervals = pd.qcut(temp['int_rate'], q=6, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'], retbins=True)


plt.figure(figsize=(15,6))
sns.barplot(data=temp, x='bucket', hue='loan_status', y='loan_amnt', estimator='sum')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['bucket','loan_status'])['id'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('bucket').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

# Set x-axis labels as the 'income_bucket' values
plt.xlabel('int_rate Bucket', fontsize=12)
plt.ylabel('Charged Off Percentage (%)', fontsize=12)

# Set title
plt.title('Charged Off Loan Percentage by int_rate Bucket', fontsize=14)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


# Get the actual bin intervals (ranges)
print("\nBin Ranges:")
for i in range(len(bin_intervals) - 1):
    print(f"Bucket {i + 1}: {bin_intervals[i]} to {bin_intervals[i + 1]}")

## Conclusion on Interest Rate

##### It can be safely concluded that loans with higher interest rates are more likely to make default

##### It also indicates that lower grade loans must be charged with higher interest rates to compensate for higher risk

## Relation between Grade and Interest Rate

##### Grade - Interest Rate Relation by Loan Count

In [ ]:
temp = df2[['id', 'grade', 'int_rate', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())
temp['int_rate'] = temp['int_rate'].str.replace("%","").astype(float)

# Remove outliers

# Calculate the IQR (Interquartile Range)
Q1 = temp['int_rate'].quantile(0.25)  # 25th percentile
Q3 = temp['int_rate'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['int_rate'] >= lower_bound) & (temp['int_rate'] <= upper_bound)]

# Define the number of unique grades
grades = np.sort(temp['grade'].unique())

# Create subplots, one for each grade
fig, axes = plt.subplots(nrows=1, ncols=len(grades), figsize=(15, 6), sharey=True)

# Loop through the grades and create a boxplot for each grade in a separate subplot
for i, grade in enumerate(grades):
    sns.boxplot(x='grade', y='int_rate', data=temp[temp['grade'] == grade], ax=axes[i])
    axes[i].set_title(f"Grade {grade}")
    axes[i].set_xlabel('Grade')
    axes[i].set_ylabel('Interest Rate')

# Adjust the layout for better spacing
plt.tight_layout()
plt.show()

## Conclusion on relationship between Grade and Interest Rate

##### Lower Grade loans are usually offered higher interest rates to compensate for higher credit risk

## Default Rate by Annual Income

##### Annual Income - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'annual_inc', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

#sns.boxplot(temp['annual_inc'])

# Calculate the IQR (Interquartile Range)
Q1 = temp['annual_inc'].quantile(0.25)  # 25th percentile
Q3 = temp['annual_inc'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['annual_inc'] >= lower_bound) & (temp['annual_inc'] <= upper_bound)]
print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['annual_inc'].describe())

sns.boxplot(temp['annual_inc'])

# Create 4 buckets based on quartiles (percentile-based)
temp['income_bucket'], bin_intervals = pd.qcut(temp['annual_inc'], q=6, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'], retbins=True)

# visual
plt.figure(figsize=(15,6))
sns.histplot(data=temp, x='income_bucket', hue='loan_status', multiple='stack', stat='count')
plt.xticks(rotation=45)
plt.show()

# Boxplot of annual income by loan status
plt.figure(figsize=(10,6))
sns.boxplot(x='loan_status', y='annual_inc', data=temp)

# Set labels and title
plt.xlabel('Loan Status (Nominal)', fontsize=12)
plt.ylabel('Annual Income (Continuous)', fontsize=12)
plt.title('Boxplot of Annual Income by Loan Status', fontsize=14)

# Show the plot
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['income_bucket','loan_status'])['id'].count()
loan_percentage = loan_count.groupby('income_bucket').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

# Set x-axis labels as the 'income_bucket' values
plt.xlabel('Income Bucket', fontsize=12)
plt.ylabel('Charged Off Percentage (%)', fontsize=12)

# Set title
plt.title('Charged Off Loan Percentage by Income Bucket', fontsize=14)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


# Get the actual bin intervals (ranges)
print("\nBin Ranges:")
for i in range(len(bin_intervals) - 1):
    print(f"Bucket {i + 1}: {bin_intervals[i]} to {bin_intervals[i + 1]}")

##### Annual Income - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'annual_inc', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

#sns.boxplot(temp['annual_inc'])

# Calculate the IQR (Interquartile Range)
Q1 = temp['annual_inc'].quantile(0.25)  # 25th percentile
Q3 = temp['annual_inc'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['annual_inc'] >= lower_bound) & (temp['annual_inc'] <= upper_bound)]
print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['annual_inc'].describe())
sns.boxplot(temp['annual_inc'])

# Create 4 buckets based on quartiles (percentile-based)
temp['income_bucket'], bin_intervals = pd.qcut(temp['annual_inc'], q=6, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'], retbins=True)



plt.figure(figsize=(15,6))
sns.barplot(data=temp, x='income_bucket', hue='loan_status', y='loan_amnt', estimator='sum')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['income_bucket','loan_status'])['id'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('income_bucket').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

# Set x-axis labels as the 'income_bucket' values
plt.xlabel('Income Bucket', fontsize=12)
plt.ylabel('Charged Off Percentage (%)', fontsize=12)

# Set title
plt.title('Charged Off Loan Percentage by Income Bucket', fontsize=14)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


# Get the actual bin intervals (ranges)
print("\nBin Ranges:")
for i in range(len(bin_intervals) - 1):
    print(f"Bucket {i + 1}: {bin_intervals[i]} to {bin_intervals[i + 1]}")

## Conclusion on Annual Income

##### It can be safely concluded that borrowers with higher income tend to make lower defaults

## Default Rate by Debt to Income ratio (dti)

##### Debt to Income ratio (dti) - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'dti', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

#sns.boxplot(temp['annual_inc'])

# Calculate the IQR (Interquartile Range)
Q1 = temp['dti'].quantile(0.25)  # 25th percentile
Q3 = temp['dti'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['dti'] >= lower_bound) & (temp['dti'] <= upper_bound)]
print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['dti'].describe())

sns.boxplot(temp['dti'])

# Create 4 buckets based on quartiles (percentile-based)
temp['bucket'], bin_intervals = pd.qcut(temp['dti'], q=6, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'], retbins=True)

# visualisation
plt.figure(figsize=(15,6))
sns.histplot(data=temp, x='bucket', hue='loan_status', multiple='stack', stat='count')
plt.xticks(rotation=45)
plt.show()

# Boxplot of annual income by loan status
plt.figure(figsize=(10,6))
sns.boxplot(x='loan_status', y='dti', data=temp)

# Set labels and title
plt.xlabel('Loan Status (Nominal)', fontsize=12)
plt.ylabel('dti (Continuous)', fontsize=12)
plt.title('Boxplot of dti by Loan Status', fontsize=14)

# Show the plot
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['bucket','loan_status'])['id'].count()
loan_percentage = loan_count.groupby('bucket').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

# Set x-axis labels as the 'income_bucket' values
plt.xlabel('dti Bucket', fontsize=12)
plt.ylabel('Charged Off Percentage (%)', fontsize=12)

# Set title
plt.title('Charged Off Loan Percentage by dti Bucket', fontsize=14)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


# Get the actual bin intervals (ranges)
print("\nBin Ranges:")
for i in range(len(bin_intervals) - 1):
    print(f"Bucket {i + 1}: {bin_intervals[i]} to {bin_intervals[i + 1]}")

##### Debt to Income - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'dti', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

#sns.boxplot(temp['annual_inc'])

# Calculate the IQR (Interquartile Range)
Q1 = temp['dti'].quantile(0.25)  # 25th percentile
Q3 = temp['dti'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['dti'] >= lower_bound) & (temp['dti'] <= upper_bound)]
print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['dti'].describe())

sns.boxplot(temp['dti'])

# Create 4 buckets based on quartiles (percentile-based)
temp['bucket'], bin_intervals = pd.qcut(temp['dti'], q=6, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'], retbins=True)


# visualisation
plt.figure(figsize=(15,6))
sns.barplot(data=temp, x='bucket', hue='loan_status', y='loan_amnt', estimator='sum')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['bucket','loan_status'])['id'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('bucket').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

# Set x-axis labels as the 'income_bucket' values
plt.xlabel('dti Bucket', fontsize=12)
plt.ylabel('Charged Off Percentage (%)', fontsize=12)

# Set title
plt.title('Charged Off Loan Percentage by Income Bucket', fontsize=14)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


# Get the actual bin intervals (ranges)
print("\nBin Ranges:")
for i in range(len(bin_intervals) - 1):
    print(f"Bucket {i + 1}: {bin_intervals[i]} to {bin_intervals[i + 1]}")

## Conclusion on Debt to Income ratio (DTI)

##### It can be safely concluded that borrowers with higher Debt to Income ratio tend to make higher defaults

##### DTI is a leading indicator of default that represents the ability of borrower to repay the loan installments

## Relationship between Annual Income and DTI

##### Annual income - dti Relation by Loan Count

In [ ]:
temp = df2[['id', 'annual_inc', 'dti', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

# Remove the outliers from dti

# Calculate the IQR (Interquartile Range)
Q1 = temp['dti'].quantile(0.25)  # 25th percentile
Q3 = temp['dti'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['dti'] >= lower_bound) & (temp['dti'] <= upper_bound)]
print('\033[1;4m' + '\ndti Statistics\n' + '\033[0m')
print(temp['dti'].describe())

sns.boxplot(temp['dti'])
plt.show()


# Remove the outliers from annual_inc

# Calculate the IQR (Interquartile Range)
Q1 = temp['annual_inc'].quantile(0.25)  # 25th percentile
Q3 = temp['annual_inc'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['annual_inc'] >= lower_bound) & (temp['annual_inc'] <= upper_bound)]
print('\033[1;4m' + '\nannual_inc Statistics\n' + '\033[0m')
print(temp['annual_inc'].describe())

sns.boxplot(temp['annual_inc'])
plt.show()


# Calculate the correlation between 'annual_inc' and 'dti'
correlation = temp['annual_inc'].corr(temp['dti'])

print('\033[1;4m' + "Correlation between annual_inc and dti:", round(correlation*100,1), '%\n' + '\033[0m')

## Conclusion on relationship between Annual Income Debt to Income (DTI)

##### There is no strong correlation between Annual income and DTI

## Default Rate by Revolving line utilization rate

##### Revolving line utilization rate - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'revol_util', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())
temp['revol_util'] = temp['revol_util'].str.replace("%","").astype(float)

# Remove outliers

# Calculate the IQR (Interquartile Range)
Q1 = temp['revol_util'].quantile(0.25)  # 25th percentile
Q3 = temp['revol_util'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['revol_util'] >= lower_bound) & (temp['revol_util'] <= upper_bound)]
print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['revol_util'].describe())

sns.boxplot(temp['revol_util'])

# Create 4 buckets based on quartiles (percentile-based)
temp['bucket'], bin_intervals = pd.qcut(temp['revol_util'], q=6, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'], retbins=True)


# visualisation
plt.figure(figsize=(15,6))
sns.histplot(data=temp, x='bucket', hue='loan_status', multiple='stack', stat='count')
plt.xticks(rotation=45)
plt.show()

# Boxplot of by loan status
plt.figure(figsize=(10,6))
sns.boxplot(x='loan_status', y='revol_util', data=temp)

# Set labels and title
plt.xlabel('Loan Status (Nominal)', fontsize=12)
plt.ylabel('revol_util (Continuous)', fontsize=12)
plt.title('Boxplot of revol_util by Loan Status', fontsize=14)

# Show the plot
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['bucket','loan_status'])['id'].count()
loan_percentage = loan_count.groupby('bucket').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

# Set x-axis labels as the 'bucket' values
plt.xlabel('revol_util Bucket', fontsize=12)
plt.ylabel('Charged Off Percentage (%)', fontsize=12)

# Set title
plt.title('Charged Off Loan Percentage by revol_util Bucket', fontsize=14)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


# Get the actual bin intervals (ranges)
print("\nBin Ranges:")
for i in range(len(bin_intervals) - 1):
    print(f"Bucket {i + 1}: {bin_intervals[i]} to {bin_intervals[i + 1]}")

##### Revolving line utilization rate - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'revol_util', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())
temp['revol_util'] = temp['revol_util'].str.replace("%","").astype(float)

# Remove outliers

# Calculate the IQR (Interquartile Range)
Q1 = temp['revol_util'].quantile(0.25)  # 25th percentile
Q3 = temp['revol_util'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['revol_util'] >= lower_bound) & (temp['revol_util'] <= upper_bound)]
print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['revol_util'].describe())

sns.boxplot(temp['revol_util'])

# Create 4 buckets based on quartiles (percentile-based)
temp['bucket'], bin_intervals = pd.qcut(temp['revol_util'], q=6, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'], retbins=True)


# visualisation
plt.figure(figsize=(15,6))
sns.barplot(data=temp, x='bucket', hue='loan_status', y='loan_amnt', estimator='sum')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['bucket','loan_status'])['id'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('bucket').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

# Set x-axis labels as the 'bucket' values
plt.xlabel('revol_util Bucket', fontsize=12)
plt.ylabel('Charged Off Percentage (%)', fontsize=12)

# Set title
plt.title('Charged Off Loan Percentage by revol_util Bucket', fontsize=14)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


# Get the actual bin intervals (ranges)
print("\nBin Ranges:")
for i in range(len(bin_intervals) - 1):
    print(f"Bucket {i + 1}: {bin_intervals[i]} to {bin_intervals[i + 1]}")

## Conclusion on Revolving line utilization rate

##### Borrowers with higher Revolving line utilization rate tend to make higher defaults

##### Borrowers may have revolving lines available for their disposal, and they tend to utilize these lines to pay off their debts in times of crisis or before making any default

##### High utilization rate is a leading indicator of likelihood of default

## Default Rate by inq_last_6mths

##### inq_last_6mths - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'inq_last_6mths', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

plt.figure(figsize=(15,6))
sns.histplot(data=temp, x='inq_last_6mths', hue='loan_status', multiple='stack', stat='count')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['inq_last_6mths','loan_status'])['id'].count()
loan_percentage = loan_count.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

##### inq_last_6mths - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'inq_last_6mths', 'loan_status', 'loan_amnt']]

plt.figure(figsize=(15,6))
sns.barplot(data=temp, x='inq_last_6mths', hue='loan_status', y='loan_amnt', estimator='sum')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['inq_last_6mths','loan_status'])['loan_amnt'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('inq_last_6mths').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

## Conclusion on inq_last_6mths

##### As a general trend, inq_last_6mths seems to be an indicator of higher default rate. This indicates that for low quality loans, borrower must have attempted for loan through multiple sources

##### Multiple inquiries also suggest that loans for such borrowers must have been rejected due to some defect by multiple lenders, which also indicate higher chances of default

## Default Rate by Purpose of loan

##### purpose - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'purpose', 'loan_status', 'loan_amnt']]

plt.figure(figsize=(15,6))
sns.histplot(data=temp, x='purpose', hue='loan_status', multiple='stack', stat='count')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Amount
loan_count = temp.groupby(['purpose','loan_status'])['id'].count()
loan_percentage = loan_count.groupby('purpose').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)

##### purpose - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'purpose', 'loan_status', 'loan_amnt']]

plt.figure(figsize=(15,6))
sns.barplot(data=temp, x='purpose', hue='loan_status', y='loan_amnt', estimator='sum')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['purpose','loan_status'])['loan_amnt'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('purpose').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)

## Conclusion on purpose

##### Small business loans have significantly high default rates

##### Disproportionally large amount of loans are made for purpose of debt consolidation

##### Consumer loan such as purchases, car, credit card, wedding, etc have relatively lower default rates

##### Such loans are typically optional in nature and tend to be taken by borrowers with expected cashflows in future to repay the loanan

## Default Rate by Installment

##### Installment - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'installment', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

# Remove outliers

# Calculate the IQR (Interquartile Range)
Q1 = temp['installment'].quantile(0.25)  # 25th percentile
Q3 = temp['installment'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['installment'] >= lower_bound) & (temp['installment'] <= upper_bound)]
print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['installment'].describe())

sns.boxplot(temp['installment'])

# Create 4 buckets based on quartiles (percentile-based)
temp['bucket'], bin_intervals = pd.qcut(temp['installment'], q=6, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'], retbins=True)

# visualisation
plt.figure(figsize=(15,6))
sns.histplot(data=temp, x='bucket', hue='loan_status', multiple='stack', stat='count')
plt.xticks(rotation=45)
plt.show()

# Boxplot of by loan status
plt.figure(figsize=(10,6))
sns.boxplot(x='loan_status', y='installment', data=temp)

# Set labels and title
plt.xlabel('Loan Status (Nominal)', fontsize=12)
plt.ylabel('installment (Continuous)', fontsize=12)
plt.title('Boxplot of installment by Loan Status', fontsize=14)

# Show the plot
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['bucket','loan_status'])['id'].count()
loan_percentage = loan_count.groupby('bucket').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

# Set x-axis labels as the 'bucket' values
plt.xlabel('installment Bucket', fontsize=12)
plt.ylabel('Charged Off Percentage (%)', fontsize=12)

# Set title
plt.title('Charged Off Loan Percentage by installment Bucket', fontsize=14)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


# Get the actual bin intervals (ranges)
print("\nBin Ranges:")
for i in range(len(bin_intervals) - 1):
    print(f"Bucket {i + 1}: {bin_intervals[i]} to {bin_intervals[i + 1]}")

##### Installment - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'installment', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

# Remove outliers

# Calculate the IQR (Interquartile Range)
Q1 = temp['installment'].quantile(0.25)  # 25th percentile
Q3 = temp['installment'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['installment'] >= lower_bound) & (temp['installment'] <= upper_bound)]
print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['installment'].describe())

sns.boxplot(temp['installment'])

# Create 4 buckets based on quartiles (percentile-based)
temp['bucket'], bin_intervals = pd.qcut(temp['installment'], q=6, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'], retbins=True)

# View the new column
print(temp[['installment', 'bucket']].head())


plt.figure(figsize=(15,6))
sns.barplot(data=temp, x='bucket', hue='loan_status', y='loan_amnt', estimator='sum')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['bucket','loan_status'])['id'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('bucket').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

# Set x-axis labels as the 'bucket' values
plt.xlabel('installment Bucket', fontsize=12)
plt.ylabel('Charged Off Percentage (%)', fontsize=12)

# Set title
plt.title('Charged Off Loan Percentage by installment Bucket', fontsize=14)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


# Get the actual bin intervals (ranges)
print("\nBin Ranges:")
for i in range(len(bin_intervals) - 1):
    print(f"Bucket {i + 1}: {bin_intervals[i]} to {bin_intervals[i + 1]}")

## Conclusion on Installment

##### Beyond a certain threshold of \~200 USD, chances of default tend to increase with the installment size

##### Larger installments increases the burden on borrower, and may be subject to default in case of any adverse condition on borrower income

## Default Rate by Length of Employment

##### emp_length - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'emp_length', 'loan_status', 'loan_amnt']]
sns.histplot(data=temp, x='emp_length', hue='loan_status', multiple='stack', stat='count')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['emp_length','loan_status'])['id'].count()
loan_percentage = loan_count.groupby('emp_length').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)

##### emp_length - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'emp_length', 'loan_status', 'loan_amnt']]
sns.barplot(data=temp, x='emp_length', hue='loan_status', y='loan_amnt', estimator='sum')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['emp_length','loan_status'])['loan_amnt'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('emp_length').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)

## Conclusion on emp_length

##### Length of employment does not seem to have any correlation with tendency to default

## Default Rate by verification_status

##### verification_status - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'verification_status', 'loan_status', 'loan_amnt']]
sns.histplot(data=temp, x='verification_status', hue='loan_status', multiple='stack', stat='count')
plt.show()

# Default rate by Loan Amount
loan_count = temp.groupby(['verification_status','loan_status'])['id'].count()
loan_percentage = loan_count.groupby('verification_status').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)

##### verification_status - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'verification_status', 'loan_status', 'loan_amnt']]
sns.barplot(data=temp, x='verification_status', hue='loan_status', y='loan_amnt', estimator='sum')
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['verification_status','loan_status'])['loan_amnt'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('verification_status').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)

## Conclusion on verification_status
##### Verification status does not seem to have any correlation with tendency to default

## Default Rate by home_ownership

##### home_ownership - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'home_ownership', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

plt.figure(figsize=(15,6))
sns.histplot(data=temp, x='home_ownership', hue='loan_status', multiple='stack', stat='count')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['home_ownership','loan_status'])['id'].count()
loan_percentage = loan_count.groupby('home_ownership').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)

##### home_ownership - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'home_ownership', 'loan_status', 'loan_amnt']]

plt.figure(figsize=(15,6))
sns.barplot(data=temp, x='home_ownership', hue='loan_status', y='loan_amnt', estimator='sum')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['home_ownership','loan_status'])['loan_amnt'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('home_ownership').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)

## Conclusion on home_ownership

##### home_ownership status does not seem to have any correlation with tendency to default

## Default Rate by addr_state

##### addr_state - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'addr_state', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

plt.figure(figsize=(15,6))
sns.histplot(data=temp, x='addr_state', hue='loan_status', multiple='stack', stat='count')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['addr_state','loan_status'])['id'].count()
loan_percentage = loan_count.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)

##### addr_state - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'addr_state', 'loan_status', 'loan_amnt']]

plt.figure(figsize=(15,6))
sns.barplot(data=temp, x='addr_state', hue='loan_status', y='loan_amnt', estimator='sum')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['addr_state','loan_status'])['loan_amnt'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('addr_state').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)

## Conclusion on addr_state

##### addr_state status does not seem to have any correlation with tendency to default

## Default Rate by delinq_2yrs

##### delinq_2yrs - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'delinq_2yrs', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

plt.figure(figsize=(15,6))
sns.histplot(data=temp, x='delinq_2yrs', hue='loan_status', multiple='stack', stat='count')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['delinq_2yrs','loan_status'])['id'].count()
loan_percentage = loan_count.groupby('delinq_2yrs').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

##### delinq_2yrs - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'delinq_2yrs', 'loan_status', 'loan_amnt']]

plt.figure(figsize=(15,6))
sns.barplot(data=temp, x='delinq_2yrs', hue='loan_status', y='loan_amnt', estimator='sum')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['delinq_2yrs','loan_status'])['loan_amnt'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('delinq_2yrs').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

## Conclusion on delinq_2yrs

##### As a general trend, although higher incidences of delinquency in 2yrs seems to be an indicator of higher default rate, but due to very limited data points in higher delinquency buckets, it is inconclusive

## Default Rate by Loan size

##### Are borrowers with higher loan_amt more likely to make default

In [ ]:
temp = df2[['id', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

#sns.boxplot(temp['annual_inc'])

# Calculate the IQR (Interquartile Range)
Q1 = temp['loan_amnt'].quantile(0.25)  # 25th percentile
Q3 = temp['loan_amnt'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['loan_amnt'] >= lower_bound) & (temp['loan_amnt'] <= upper_bound)]
print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['loan_amnt'].describe())

sns.boxplot(temp['loan_amnt'])

# Create 4 buckets based on quartiles (percentile-based)
temp['bucket'], bin_intervals = pd.qcut(temp['loan_amnt'], q=10, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10'], retbins=True)

# View the new column
print(temp[['loan_amnt', 'bucket']].head())

# visualisation
plt.figure(figsize=(15,6))
sns.histplot(data=temp, x='bucket', hue='loan_status', multiple='stack', stat='count')
plt.xticks(rotation=45)
plt.show()

# Boxplot of annual income by loan status
plt.figure(figsize=(10,6))
sns.boxplot(x='loan_status', y='loan_amnt', data=temp)

# Set labels and title
plt.xlabel('Loan Status (Nominal)', fontsize=12)
plt.ylabel('loan_amnt (Continuous)', fontsize=12)
plt.title('Boxplot of dti by Loan Status', fontsize=14)

# Show the plot
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['bucket','loan_status'])['id'].count()
loan_percentage = loan_count.groupby('bucket').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

# Set x-axis labels as the 'income_bucket' values
plt.xlabel('loan_amnt Bucket', fontsize=12)
plt.ylabel('Charged Off Percentage (%)', fontsize=12)

# Set title
plt.title('Charged Off Loan Percentage by loan_amnt Bucket', fontsize=14)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


# Get the actual bin intervals (ranges)
print("\nBin Ranges:")
for i in range(len(bin_intervals) - 1):
    print(f"Bucket {i + 1}: {bin_intervals[i]} to {bin_intervals[i + 1]}")

## Conclusion on relationship between default rate and loan size

##### For small loan amounts, no correlation seems to exist between loan amount and default rate but for loan amount higher than certain threshold (\~15,000), tendency to default increases with loan amount

## Default Rate by Total credit revolving balance

##### Total credit revolving balance - Defualt Rate on Loan Count

In [ ]:
temp = df2[['id', 'revol_bal', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

# Remove outliers

# Calculate the IQR (Interquartile Range)
Q1 = temp['revol_bal'].quantile(0.25)  # 25th percentile
Q3 = temp['revol_bal'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['revol_bal'] >= lower_bound) & (temp['revol_bal'] <= upper_bound)]
print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['revol_bal'].describe())

sns.boxplot(temp['revol_bal'])

# Create 4 buckets based on quartiles (percentile-based)
temp['bucket'], bin_intervals = pd.qcut(temp['revol_bal'], q=6, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'], retbins=True)


# visualisation
plt.figure(figsize=(15,6))
sns.histplot(data=temp, x='bucket', hue='loan_status', multiple='stack', stat='count')
plt.xticks(rotation=45)
plt.show()

# Boxplot of by loan status
plt.figure(figsize=(10,6))
sns.boxplot(x='loan_status', y='revol_bal', data=temp)

# Set labels and title
plt.xlabel('Loan Status (Nominal)', fontsize=12)
plt.ylabel('revol_bal (Continuous)', fontsize=12)
plt.title('Boxplot of revol_util by Loan Status', fontsize=14)

# Show the plot
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Default rate by Loan Count
loan_count = temp.groupby(['bucket','loan_status'])['id'].count()
loan_percentage = loan_count.groupby('bucket').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Count\n' + '\033[0m')
print(loan_count)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

# Set x-axis labels as the 'bucket' values
plt.xlabel('revol_bal Bucket', fontsize=12)
plt.ylabel('Charged Off Percentage (%)', fontsize=12)

# Set title
plt.title('Charged Off Loan Percentage by revol_bal Bucket', fontsize=14)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


# Get the actual bin intervals (ranges)
print("\nBin Ranges:")
for i in range(len(bin_intervals) - 1):
    print(f"Bucket {i + 1}: {bin_intervals[i]} to {bin_intervals[i + 1]}")

##### Total credit revolving balance - Default Rate on Loan Amount

In [ ]:
temp = df2[['id', 'revol_bal', 'loan_status', 'loan_amnt']]
print(temp.isnull().sum())

# Remove outliers

# Calculate the IQR (Interquartile Range)
Q1 = temp['revol_bal'].quantile(0.25)  # 25th percentile
Q3 = temp['revol_bal'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['revol_bal'] >= lower_bound) & (temp['revol_bal'] <= upper_bound)]
print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['revol_bal'].describe())

sns.boxplot(temp['revol_bal'])

# Create 4 buckets based on quartiles (percentile-based)
temp['bucket'], bin_intervals = pd.qcut(temp['revol_bal'], q=6, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'], retbins=True)

# visualisation
plt.figure(figsize=(15,6))
sns.barplot(data=temp, x='bucket', hue='loan_status', y='loan_amnt', estimator='sum')
plt.xticks(rotation=45)
plt.show()

# Default rate by Loan Amount
loan_amnt_sum = temp.groupby(['bucket','loan_status'])['id'].sum()
loan_amnt_percentage = loan_amnt_sum.groupby('bucket').apply(lambda x: 100 * x / float(x.sum()))
charged_off_percentage = loan_amnt_percentage.xs('Charged Off', level='loan_status')
print('\033[1;4m' + '\nLoan Status by Loan Amount\n' + '\033[0m')
print(loan_amnt_sum)
print('\033[1;4m' + '\nDefault Rate (in percentage)\n' + '\033[0m')
print(charged_off_percentage)
charged_off_percentage.plot(kind='line', marker='o', linestyle='-', color='b', linewidth=2)

# Set x-axis labels as the 'bucket' values
plt.xlabel('revol_bal Bucket', fontsize=12)
plt.ylabel('Charged Off Percentage (%)', fontsize=12)

# Set title
plt.title('Charged Off Loan Percentage by revol_bal Bucket', fontsize=14)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


# Get the actual bin intervals (ranges)
print("\nBin Ranges:")
for i in range(len(bin_intervals) - 1):
    print(f"Bucket {i + 1}: {bin_intervals[i]} to {bin_intervals[i + 1]}")

## Conclusion on Total credit revolving balance

##### No strong correlation of default behaviour is observed with revolving balance

## Recovery Rate by Grade

In [ ]:
temp = df2[['id', 'grade', 'loan_status', 'loan_amnt', 'total_rec_prncp', 'recoveries']]
temp = temp[temp['loan_status']=='Charged Off']
print(temp.isnull().sum())
temp['recovery_rate'] = temp['recoveries'] / (temp['loan_amnt'] - temp['total_rec_prncp'])*100

print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
temp['recovery_rate'].describe()


# Remove outliers

# Calculate the IQR (Interquartile Range)
Q1 = temp['recovery_rate'].quantile(0.25)  # 25th percentile
Q3 = temp['recovery_rate'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['recovery_rate'] >= lower_bound) & (temp['recovery_rate'] <= upper_bound)]
print(temp['recovery_rate'].describe())

# Define the number of unique grades
grades = np.sort(temp['grade'].unique())
print(grades)

# Create subplots, one for each grade
fig, axes = plt.subplots(nrows=1, ncols=len(grades), figsize=(15, 6), sharey=True)

# Loop through the grades and create a boxplot for each grade in a separate subplot
for i, grade in enumerate(grades):
    sns.boxplot(x='grade', y='recovery_rate', data=temp[temp['grade'] == grade], ax=axes[i])
    axes[i].set_title(f"Grade {grade}")
    axes[i].set_xlabel('Grade')
    axes[i].set_ylabel('Recovery Rate')

# Adjust the layout for better spacing
plt.tight_layout()
plt.show()

## Conclusion on relationship between Grade and Interest Rate

##### No correlation of recovery rate is observed with grade

## Recovery Rate by Purpose

In [ ]:
temp = df2[['id', 'purpose', 'loan_status', 'loan_amnt', 'total_rec_prncp', 'recoveries']]
temp = temp[temp['loan_status']=='Charged Off']
print(temp.isnull().sum())
temp['recovery_rate'] = temp['recoveries'] / (temp['loan_amnt'] - temp['total_rec_prncp'])*100

print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
temp['recovery_rate'].describe()


# Remove outliers

# Calculate the IQR (Interquartile Range)
Q1 = temp['recovery_rate'].quantile(0.25)  # 25th percentile
Q3 = temp['recovery_rate'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['recovery_rate'] >= lower_bound) & (temp['recovery_rate'] <= upper_bound)]
print(temp['recovery_rate'].describe())

# Define the number of unique purpose
purposes = np.sort(temp['purpose'].unique())

# Create subplots, one for each purpose
fig, axes = plt.subplots(nrows=1, ncols=len(purposes), figsize=(15, 6), sharey=True)

# Loop through the purposes and create a boxplot for each purpose in a separate subplot
for i, purpose in enumerate(purposes):
    sns.boxplot(x='purpose', y='recovery_rate', data=temp[temp['purpose'] == purpose], ax=axes[i])

    axes[i].set_xlabel('Purpose')
    axes[i].set_ylabel('Recovery Rate')

# Adjust the layout for better spacing
plt.tight_layout()
plt.show()

## Conclusion on relationship between Purpose and Recovery Rate

##### Although recovery rate appears relatively higher for renewables and moving purpose, but no strong correlation of recovery rate is observed with Purpose

## Recovery Rate by Home Ownership

In [ ]:
temp = df2[['id', 'home_ownership', 'loan_status', 'loan_amnt', 'total_rec_prncp', 'recoveries']]
temp = temp[temp['loan_status']=='Charged Off']
print(temp.isnull().sum())
temp['recovery_rate'] = temp['recoveries'] / (temp['loan_amnt'] - temp['total_rec_prncp'])*100
temp['recovery_rate'].describe()


# Remove outliers

# Calculate the IQR (Interquartile Range)
Q1 = temp['recovery_rate'].quantile(0.25)  # 25th percentile
Q3 = temp['recovery_rate'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['recovery_rate'] >= lower_bound) & (temp['recovery_rate'] <= upper_bound)]

print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['recovery_rate'].describe())

# Define the number of unique home_ownership
ho = np.sort(temp['home_ownership'].unique())
print(ho)

# Create subplots, one for each home_ownership
fig, axes = plt.subplots(nrows=1, ncols=len(ho), figsize=(15, 6), sharey=True)

# Loop through the home_ownership and create a boxplot for each home_ownership in a separate subplot
for i, x in enumerate(ho):
    sns.boxplot(x='home_ownership', y='recovery_rate', data=temp[temp['home_ownership'] == x], ax=axes[i])
    axes[i].set_title(f"Home Ownership {x}")
    axes[i].set_xlabel('Home Ownership')
    axes[i].set_ylabel('Recovery Rate')

# Adjust the layout for better spacing
plt.tight_layout()
plt.show()

## Conclusion on relationship between Home Ownership and Recovery Rate

##### Recovery Rate is significantly lower for ‘OTHER’ home ownership category. But due to very limited data points for ‘OTHER’ category, there is no enough evidence to conclude anything

## Recovery Rate by Annual Income

In [ ]:
temp = df2[['id', 'annual_inc', 'loan_status', 'loan_amnt', 'total_rec_prncp', 'recoveries']]
temp = temp[temp['loan_status']=='Charged Off']
print(temp.isnull().sum())
temp['recovery_rate'] = temp['recoveries'] / (temp['loan_amnt'] - temp['total_rec_prncp'])*100

# Remove outliers for Annual Income

# Calculate the IQR (Interquartile Range)
Q1 = temp['annual_inc'].quantile(0.25)  # 25th percentile
Q3 = temp['annual_inc'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['annual_inc'] >= lower_bound) & (temp['annual_inc'] <= upper_bound)]

print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['annual_inc'].describe())


# Remove outliers for Recovery Rate

# Calculate the IQR (Interquartile Range)
Q1 = temp['recovery_rate'].quantile(0.25)  # 25th percentile
Q3 = temp['recovery_rate'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['recovery_rate'] >= lower_bound) & (temp['recovery_rate'] <= upper_bound)]

print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['recovery_rate'].describe())

# Find Correlation
print('\033[1;4m' + '\nCorrelation between Annual Income and Recovery Rate is ' + str(round(temp['recovery_rate'].corr(temp['annual_inc'])*100)) + "%" + '\033[0m')

## Conclusion on relationship between Annual Income and Recovery Rate

##### No correlation exists between annual income and recovery rate

#### Recovery Rate by Verification Status

In [ ]:
temp = df2[['id', 'verification_status', 'loan_status', 'loan_amnt', 'total_rec_prncp', 'recoveries']]
temp = temp[temp['loan_status']=='Charged Off']
print(temp.isnull().sum())
temp['recovery_rate'] = temp['recoveries'] / (temp['loan_amnt'] - temp['total_rec_prncp'])*100
temp['recovery_rate'].describe()


# Remove outliers

# Calculate the IQR (Interquartile Range)
Q1 = temp['recovery_rate'].quantile(0.25)  # 25th percentile
Q3 = temp['recovery_rate'].quantile(0.75)  # 75th percentile
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
temp = temp[(temp['recovery_rate'] >= lower_bound) & (temp['recovery_rate'] <= upper_bound)]

print('\033[1;4m' + '\nVariable Statistics\n' + '\033[0m')
print(temp['recovery_rate'].describe())

# Define the number of unique verification_status
vs = np.sort(temp['verification_status'].unique())
print(vs)

# Create subplots, one for each verification_status
fig, axes = plt.subplots(nrows=1, ncols=len(vs), figsize=(15, 6), sharey=True)

# Loop through the verification_status and create a boxplot for each grade in a separate subplot
for i, x in enumerate(vs):
    sns.boxplot(x='verification_status', y='recovery_rate', data=temp[temp['verification_status'] == x], ax=axes[i])
    axes[i].set_title(f"Verification Status {x}")
    axes[i].set_xlabel('Verification Status')
    axes[i].set_ylabel('Recovery Rate')

# Adjust the layout for better spacing
plt.tight_layout()
plt.show()

## Conclusion on relationship between Verification Status and Recovery Rate

##### No correlation of recovery rate is observed with Verification Status